In [ ]:
# %pip install tokenizers

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ruediste/codeparrot-github-code-10G", "cs", split="train")
dataset


In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from transformers import PreTrainedTokenizerFast

tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.decoder=decoders.ByteLevel()

tokenizer.clean_up_tokenization_spaces=True

special_tokens = ["<|pad|>", "<|mask|>", "<|cls|>"]
tokenizer.add_special_tokens(special_tokens)
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=special_tokens,max_token_length=10, show_progress=True)

def datasetGenerator():
    for row in dataset.take(5000):
        yield row['code']

tokenizer.train_from_iterator(datasetGenerator(), trainer=trainer)

hft=PreTrainedTokenizerFast(pad_token='<|pad|>', cls_token='<|cls|>', mask_token='<|mask|>',sep_token='<|pad|>' tokenizer_object=tokenizer )
hft.save_pretrained('data/bert1')
tokenizer=hft

In [ ]:
ids=tokenizer("foo.bar(123); hello world")["input_ids"]
print(ids)
print(tokenizer.convert_ids_to_tokens(ids))
tokenizer.decode(ids)
